In [19]:
from pathlib import Path
import pprint
import os


__file__ = os.getcwd()
data_paths = [Path(__file__) / 'data' / 'dbCAN2' / 'CAZyDB.07312018.fa',
             Path(__file__) / 'data' / 'dbCAN2' / 'CAZyDB.07312019.fa',
             Path(__file__) / 'data' / 'dbCAN2' / 'CAZyDB.07312020.fa']

pprint.pprint(data_paths)

path:
['/home/sunmoon/workspace/eCAMI/data/dbCAN2/CAZyDB.07312018.fa',
 '/home/sunmoon/workspace/eCAMI/data/dbCAN2/CAZyDB.07312019.fa',
 '/home/sunmoon/workspace/eCAMI/data/dbCAN2/CAZyDB.07312020.fa']


In [12]:
from tqdm import tqdm
import pandas as pd
import hashlib


class Protein:
    def __init__(self, name='', families='', sequence='', sequence_hash=''):
        self.name = name
        self.families = families
        self.sequence = sequence
        self.sequence_hash = sequence_hash
        
    def hash_sequence(self):
        self.sequence_hash = hashlib.sha1(self.sequence.encode('ascii')).hexdigest()

    def to_list(self):
        return [self.name, self.families, self.sequence, self.sequence_hash]
        
    def __repr__(self):
        return f'''\r{self.name}
        \r{self.families}
        \r{self.sequence[:20]}...{len(self.sequence)}...{self.sequence[-5:]}
        \r{self.sequence_hash}'''


proteins = []
temp_protein = None

for data_path in data_paths:
    data_file = open(data_path, 'r')

    for line in tqdm(data_file):
        line = line.strip().upper()

        if line[-1] == '|':
            line = line[:-1]
        
        if line[0] == '>':
            if temp_protein is not None:
                temp_protein.hash_sequence()
                proteins.append(temp_protein.to_list())
            
            temp_protein = Protein()
            name, families = line.split('|', 1)
            # try:
            #     name, families = line.split('|', 1)
            # except:
            #     print(f'\n[ERROR] format error: {line}')
            #     print(name, families)
            
            name = name[1:]
            
            temp_protein.name = name
            temp_protein.families = families
            
        else:
            temp_protein.sequence += line

columns = ['name', 'families', 'sequence', 'sequence_hash']
protein_df = pd.DataFrame(proteins, columns=columns)
print(f'Protein DF shape: {protein_df.shape}')

protein_df = protein_df.drop_duplicates(subset=['sequence_hash'], keep='first')
print(f'Unique protein DF shape: {protein_df.shape}')

2134124it [00:05, 420644.30it/s]
2775168it [00:10, 265222.74it/s]
367475it [00:00, 411981.00it/s][ERROR] format error
>QJT90597.2
AMU39661.1 GT4
[ERROR] format error
>EFD01313.1
AII35530.1 GT28
703251it [00:01, 414951.85it/s][ERROR] format error
>QEO52270.2
ADY01250.1 GT4
[ERROR] format error
>QEP53074.2
AHB09681.1 GT51
829093it [00:02, 418438.24it/s][ERROR] format error
>AXM04939.2
QKY19549.1 GT4
[ERROR] format error
>XP_006205874.2
ALJ01267.1 GH3
1082803it [00:02, 422414.19it/s][ERROR] format error
>AMG24085.2
AGN83749.1 GH23
1379583it [00:03, 423318.76it/s][ERROR] format error
>XP_002188159.4
AQR77870.1 CBM13
1504867it [00:05, 89096.30it/s][ERROR] format error
>XP_003906761.2
AOA45118.1 GT4
1668527it [00:05, 217507.23it/s][ERROR] format error
>XP_011420931.2
AAQ21404.1 GH18
[ERROR] format error
>XP_001818405.3
ANR89767.1 CBM5|GH18
2039486it [00:06, 400123.92it/s][ERROR] format error
>QJT88798.2
QCZ74525.1 GT4
2328063it [00:07, 407973.98it/s][ERROR] format error
>BBI74939.2
BBX81405.

In [13]:
from tqdm import tqdm
import pickle
import pprint


def append_key(family_dict, family_levels, protein):
    total_levels = len(family_levels)
    
    cw_dict = family_dict
    
    for i in range(total_levels):
        if family_levels[i] not in cw_dict:
            cw_dict[family_levels[i]] = {
                '_count' : 0,
                '_sub_count' : 1,
                '_elements' : [],
            }
            
        else:
            cw_dict[family_levels[i]]['_sub_count'] += 1
            
        cw_dict = cw_dict[family_levels[i]]
        
        if i == total_levels-1 :
            cw_dict['_count'] += 1
            cw_dict['_elements'].append(protein)
            

def append_key_count(family_dict, family_levels):
    total_levels = len(family_levels)
    
    cw_dict = family_dict
    
    for i in range(total_levels):
        if family_levels[i] not in cw_dict:
            cw_dict[family_levels[i]] = {
                '_count' : 0,
                '_sub_count' : 1,
            }
            
        else:
            cw_dict[family_levels[i]]['_sub_count'] += 1
            
        cw_dict = cw_dict[family_levels[i]]
        
        if i == total_levels-1 :
            cw_dict['_count'] += 1

            
family_dict = {}
family_count_dict = {}

for _, protein in tqdm(protein_df.iterrows()):
    family_levels_list = [family.split('_') for family in protein['families'].split('|')]

    for family_levels in family_levels_list:
        append_key(family_dict, family_levels, Protein(name=protein['name'],
                                                       families=family_levels_list,
                                                       sequence=protein['sequence'],
                                                       sequence_hash=protein['sequence_hash']))
        append_key_count(family_count_dict, family_levels)

pprint.pprint(family_count_dict)

322587it [00:30, 10687.45it/s]


KeyboardInterrupt: 

In [9]:
out_filename = 'family.pickle'
out_file = open(out_filename, 'wb')
pickle.dump(family_dict, out_file)
out_file.close()

In [10]:
out_filename = 'family_count.pickle'
out_file = open(out_filename, 'wb')
pickle.dump(family_count_dict, out_file)
out_file.close()